In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# Importing the test database
twitter_df = pd.read_csv("test_database.csv")
twitter_df

,Unnamed: 0,text,dana_sentiment,dana_sarcasm,david_sentiment,david_sarcasm,keerti_sentiment,keerti_sarcasm,kevin_sentiment,kevin_sarcasm,sentiment_conflict,output
0,0,My daughters and I voted today for you....did ...,anti-gun,NaN,NaN,NaN,anti-gun,not sarcastic,NaN,NaN,NaN,anti-gun
1,1,SHAME on YOU @GregAbbott_TX ‼️How do you sleep...,anti-gun,NaN,NaN,NaN,anti-gun,not sarcastic,NaN,NaN,NaN,anti-gun
2,2,Greg Abbott can’t and won’t keep your kids saf...,anti-gun,NaN,NaN,NaN,anti-gun,not sarcastic,NaN,NaN,NaN,anti-gun
3,3,Caitlyne shares a name with my youngest\r\n\r\...,anti-gun,NaN,NaN,NaN,anti-gun,not sarcastic,NaN,NaN,NaN,anti-gun
4,4,Like millions of @MomsAGAbbott &amp; @MomsDema...,anti-gun,NaN,NaN,NaN,anti-gun,not sarcastic,NaN,NaN,NaN,anti-gun
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,St. Louis County mayors join Valentine in call...,NaN,NaN,anti-gun,not sarcastic,NaN,NaN,anti-gun,not sarcastic,NaN,anti-gun
96,96,#GunControlNow #GunControl It’s past due https...,NaN,NaN,anti-gun,not sarcastic,NaN,NaN,anti-gun,not sarcastic,NaN,anti-gun
97,97,@GalenMicheal In other parts of the world it t...,NaN,NaN,anti-gun,not sarcastic,NaN,NaN,anti-gun,not sarcastic,NaN,anti-gun
98,98,"What's #Measure114, you may wonder?\r\n\r\nMea...",NaN,NaN,neutral,not sarcastic,NaN,NaN,neutral,not sarcastic,NaN,neutral


In [5]:
# Drop irrelevant columns
twitter_df = twitter_df.drop(['dana_sentiment', 'dana_sarcasm', 'david_sentiment', 'david_sarcasm', 'keerti_sentiment', 'keerti_sarcasm', 'kevin_sentiment', 'kevin_sarcasm', 'sentiment_conflict'], axis=1)
twitter_df

,Unnamed: 0,text,output
0,0,My daughters and I voted today for you....did ...,anti-gun
1,1,SHAME on YOU @GregAbbott_TX ‼️How do you sleep...,anti-gun
2,2,Greg Abbott can’t and won’t keep your kids saf...,anti-gun
3,3,Caitlyne shares a name with my youngest\r\n\r\...,anti-gun
4,4,Like millions of @MomsAGAbbott &amp; @MomsDema...,anti-gun
...,...,...,...
95,95,St. Louis County mayors join Valentine in call...,anti-gun
96,96,#GunControlNow #GunControl It’s past due https...,anti-gun
97,97,@GalenMicheal In other parts of the world it t...,anti-gun
98,98,"What's #Measure114, you may wonder?\r\n\r\nMea...",neutral


In [6]:
# Data information
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   text        100 non-null    object
 2   output      100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [7]:
# Length of the dataset
print("Length of the data is", len(twitter_df))

Length of the data is 100


In [8]:
# Checking for null values
np.sum(twitter_df.isnull().any(axis=1))

0

In [9]:
# Calculate the count of different sentiment types
sentiment_counts = twitter_df.groupby(['output']).size()
print(sentiment_counts)

output
anti-gun    58
neutral     25
pro-gun     17
dtype: int64
